In [1]:
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth vllm
!pip install triton==3.1.0
!pip install -U pynvml

cryptography 44.0.2 is installed but cryptography<44 is required
Redoing requirement with just package name...
pyOpenSSL 25.0.0 is installed but pyOpenSSL<=24.2.1,>=19.1.0 is required
Redoing requirement with just package name...
rich 14.0.0 is installed but rich<14,>=12.4.4 is required
Redoing requirement with just package name...
fsspec 2025.3.2 is installed but fsspec[http]<=2024.12.0,>=2023.1.0 is required
Redoing requirement with just package name...
gymnasium 0.29.0 is installed but gymnasium>=1.0.0 is required
Redoing requirement with just package name...
fsspec 2025.3.2 is installed but fsspec==2024.10.0 is required
Redoing requirement with just package name...
scipy 1.15.2 is installed but scipy<1.14.0,>=1.7.0 is required
Redoing requirement with just package name...
google-api-core 1.34.1 is installed but google-api-core[grpc]<3.0.0dev,>=2.16.0 is required
Redoing requirement with just package name...
notebook 6.5.4 is installed but notebook==6.5.5 is required
Redoing require

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_Token")

In [ ]:
!rm state.db

In [4]:
#from unsloth import FastLanguageModel, is_bfloat16_supported
#import torch
#max_seq_length = 512 # Can increase for longer reasoning traces
#lora_rank = 16 # Larger rank = smarter, but slower

#model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "google/txgemma-9b-predict",
    #token = secret_value_0,
    #max_seq_length = max_seq_length,
    #load_in_4bit = True, # False for LoRA 16bit
    #fast_inference = True, # Enable vLLM fast inference
    #trust_remote_code = False,    # ← allow the repo’s own tokenizer code
    #fix_tokenizer = True,    # ← apply Unsloth’s Gemma tokenizer patches
    #max_lora_rank = lora_rank,
    #gpu_memory_utilization = 0.8, # Reduce if out of memory
#)

#model = FastLanguageModel.get_peft_model(
    #model,
    #r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    #target_modules = ["gate_proj", "up_proj", "down_proj",],
    #lora_alpha = lora_rank,
    #use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    #random_state = 3407,
#)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/txgemma-9b-predict"

quant_config = BitsAndBytesConfig(
    load_in_4bit            = True,
    bnb_4bit_quant_type     = "nf4",
    bnb_4bit_compute_dtype  = torch.float16,
    llm_int8_enable_fp32_cpu_offload = True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id,token=secret_value_0)  # loads tokenizer.json, tokenizer.model, etc. :contentReference[oaicite:0]{index=0}
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token = secret_value_0,
    quantization_config = quant_config,
    device_map          = "balanced",      # shards weights over both GPUs if available
    torch_dtype         = torch.float16,
    attn_implementation = "eager",     # match Google’s example
)

#import torch
#from transformers import (
    #AutoTokenizer,
    #AutoModelForCausalLM,
    #BitsAndBytesConfig,
#)

#model_id = "google/txgemma-9b-predict"

# 1) Set up 4-bit NF4 + FP16 compute + CPU offload
#quant_config = BitsAndBytesConfig(
    #load_in_4bit                     = True,
    #bnb_4bit_quant_type              = "nf4",
    #bnb_4bit_compute_dtype           = torch.float16,
    #llm_int8_enable_fp32_cpu_offload = True,
#)

# 2) Load tokenizer & model, forcing "local only" so we never hit the network
#tokenizer = AutoTokenizer.from_pretrained(
    #model_id,
    #use_fast=True,
    #local_files_only=True,            # ↪ only read from ~/.cache/huggingface
#)

#model = AutoModelForCausalLM.from_pretrained(
    #model_id,
    #quantization_config = quant_config,
    #device_map          = "balanced",   # ↪ spread layers across GPU0 & GPU1
    #torch_dtype         = torch.float16,
    #local_files_only    = True,         # ↪ only read from cache
    #attn_implementation   = "eager",
#)


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/851 [00:00<?, ?B/s]

...

model-00002-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
!df -h
!du -sh ~/.cache/huggingface/hub

In [5]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def get_trialbench(split="train"):
    raw = load_dataset(
        "json",
        data_files="/kaggle/input/txgemma-datasets-trialbench-adverse-event/txgemma_datasets_trialbench_adverse-event-rate-prediction_train.jsonl",
        split=split,
    )
    # Concatenate the system prompt and the user text into one string
    def fmt(x):
        return {
            "prompt": SYSTEM_PROMPT.strip() + "\n\n" + x["input_text"].strip(),
            "answer": x["output_text"].strip(),
        }
    return raw.map(fmt)

dataset = get_trialbench()

def extract_xml_answer(text: str) -> str:
    # exactly as before
    answer = text.split("<answer>")[-1].split("</answer>")[0]
    return answer.strip()

def count_xml(text: str) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:        count += 0.125
    if text.count("\n</reasoning>\n") == 1:     count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1]) * 0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1) * 0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    # completions is List[str]
    return [count_xml(c) for c in completions]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    return [0.5 if re.match(pattern, c, re.DOTALL) else 0.0 for c in completions]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    return [0.5 if re.search(pattern, c, re.DOTALL) else 0.0 for c in completions]

def int_reward_func(completions, **kwargs) -> list[float]:
    extracted = [extract_xml_answer(c) for c in completions]
    return [0.5 if r.isdigit() else 0.0 for r in extracted]

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    # `answer` is List[str] of ground-truth
    extracted = [extract_xml_answer(c) for c in completions]
    return [2.0 if r == a else 0.0 for r, a in zip(extracted, answer)]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/14368 [00:00<?, ? examples/s]

In [6]:
from peft import LoraConfig, get_peft_model
# 2) *Attach* LoRA adapters with PEFT
lora_cfg = LoraConfig(
    r             = 16,
    lora_alpha    = 16,
    target_modules= ["gate_proj", "up_proj", "down_proj"],
    bias          = "none",
    task_type     = "CAUSAL_LM",
)
peft_model = get_peft_model(model, lora_cfg)

In [7]:
from trl import GRPOConfig

training_args = GRPOConfig(
    use_vllm                    = False,      # HF path, not Unsloth/vLLM
    learning_rate               = 5e-6,
    per_device_train_batch_size = 2,          # MUST match num_generations
    gradient_accumulation_steps = 3,
    num_generations             = 2,
    max_prompt_length           = 256,
    max_completion_length       = 200,
    max_steps                   = 100,
    save_steps                  = 250,
    output_dir                  = "outputs",

    # precision flags for a T4
    bf16                        = False,
    fp16                        = True,

    # (other args you already had:)
    weight_decay                = 0.1,
    warmup_ratio                = 0.1,
    lr_scheduler_type           = "cosine",
    optim                       = "paged_adamw_8bit",
    logging_steps               = 1,
    max_grad_norm               = 0.1,
    report_to                   = "none",
)


In [8]:
from trl import GRPOTrainer
trainer = GRPOTrainer(
    model = peft_model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

INFO 04-26 19:25:25 [__init__.py:239] Automatically detected platform cuda.


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-680d3ce0-2406b2d5029659537fadadb7;19c152ff-1c2a-4ac5-93a4-0a02158c3fa7)

Cannot access gated repo for url https://huggingface.co/google/txgemma-9b-predict/resolve/main/config.json.
Access to model google/txgemma-9b-predict is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in google/txgemma-9b-predict.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in google/txgemma-9b-predict - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=100, training_loss=0.0008768152229007153, metrics={'train_runtime': 2490.1785, 'train_samples_per_second': 0.241, 'train_steps_per_second': 0.04, 'total_flos': 0.0, 'train_loss': 0.0008768152229007153})

In [11]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# 1) Quant config exactly as you trained
quant_config = BitsAndBytesConfig(
    load_in_4bit                     = True,
    bnb_4bit_quant_type              = "nf4",
    bnb_4bit_compute_dtype           = torch.float16,
    llm_int8_enable_fp32_cpu_offload = False,    # for pure GPU inference
)

model_id = "google/txgemma-9b-predict"

# 2) Reload the base quantized model from your cache onto GPU0
base = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config   = quant_config,
    device_map            = {"": 0},       # everything on cuda:0
    torch_dtype           = torch.float16,
    local_files_only      = True,          # read from ~/.cache only
    attn_implementation   = "eager",
)

# 3) Re-attach the adapters from checkpoint-100
peft_model = PeftModel.from_pretrained(
    base,
    "outputs/checkpoint-100",  # <-- this folder holds adapter_model.safetensors etc.
    local_files_only=True,
)

# 4) Move to GPU0 & eval
device = torch.device("cuda:0")
peft_model.to(device).eval()

# 5) Prepare your prompt & tokenize
prompt = SYSTEM_PROMPT.strip() + "\n\nWill there be an adverse event for this trial?"
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

# 6) Generate
out_ids = peft_model.generate(
    **inputs,
    do_sample       = True,
    temperature     = 0.8,
    top_p           = 0.95,
    max_new_tokens  = 200,
    pad_token_id    = tokenizer.eos_token_id,
)

print(tokenizer.decode(out_ids[0], skip_special_tokens=True))


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

Will there be an adverse event for this trial?746


In [12]:
from tqdm import tqdm

for example in dataset.select(range(10)):
    out = peft_model.generate(
        **tokenizer(example["prompt"], return_tensors="pt").to(device),
        do_sample=False,  # greedy for clarity
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
    )
    print(tokenizer.decode(out[0], skip_special_tokens=True))

Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>

From the following information about a clinical trial, predict whether it would have an adverse event.

Title: Safety, Tolerability and Pharmacokinetics of Single and Repeat Doses of GSK2292767 in Healthy Participants Who Smoke Cigarettes
Summary: This study is the first administration of GSK2292767 to humans. The study will evaluate the safety, tolerability, pharmacokinetics (PK) and pharmacodynamics (PD) of single and repeat inhaled doses of GSK2292767 in healthy smokers. This study is intended to provide sufficient confidence in the safety of the molecule and preliminary information on target engagement to allow progression to further repeat dose and proof of mechanism studies. This is a two part, single site, randomized, double-blind (sponsor open), placebo controlled study. Part A will consist of two 3-period interlocking cohorts to evaluate the safety, tolerability and pharmacokinetics of ascend

In [ ]:
model.save_lora("txgemma_grpo_lora")

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )